# Final project for <i>How to win a Data Science competition</i>
<br>In this final project we'll be predicting itens sale from a company.
<br>The full description from this problem can be checked here: https://www.kaggle.com/c/competitive-data-science-predict-future-sales. The course can be acessed to Coursera plataform in <i>"How to win a Data Science competition"</i>
<br>This is the second and final notebook for this problem, for reference and understand what we did please check <b> Data Preparation and feature engineering </b>first.
<br>
<br> Ps: Notice that are two different "tests":
1. x_test is actually a "validation" dataset where we check for the performance in another population and diagnosis the model as a stable. 
2. The "test" dataset is the one that we're scoring without being able to check for performane and then sending to kaggle!

# Summary

* In this notebook only LGBM will be used for a very simple reason: the dataset is just huge and my hardware just can't run on XGB/CAT library. We'll also test some other models but it'll be focused on LGBM.
My strategy will be the following:
1. First of all check for "vanila" performance (no parameter tuning) for my *own* parameters (that I generally use while modeling). I'll be doing this for standard gbst and dart in LGBM library. And then:

* Extremely overfitting the data: I want to check the best rmse (or l2) that we can achieve first
<br><b>OR</b> if the model overfits for training data:
* <i>Regulating</i> our data: Trying to adjust parameters to decrease overfit but also decrease learning in the process.

In [2]:
import lightgbm as lgbm
import pandas as pd
import numpy as np
x_train=pd.read_csv('train_dfv3.csv')
x_test=pd.read_csv('test_dfv3.csv')
#test=pd.read_csv('test_final.csv')
#test.drop('Unnamed: 0', axis=1, inplace=True)
#test.fillna(0, inplace=True)

In [3]:
x_train.drop('Unnamed: 0', axis=1, inplace=True)
x_test.drop('Unnamed: 0', axis=1, inplace=True)

x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)

In [6]:
x_train.columns=['date_block_num', 'shop_id', 'item_id', 'target', 'item_price',
       'last_month_revenue', 'last_2month_avg_revenue',
       'last_3month_avg_revenue', 'last_3month_deviation',
       'item_is_featured_global', 'item_last_month_revenue_compared',
       'item_last_2month_avg_revenue_compared',
       'item_last_3month_avg_revenue_compared', 'shop_is_opening',
       'shop_was_closed_month_before', 'shop_was_closed_6_months_before',
       'last_2month_revenue', 'item_last_2month_revenue_compared',
       'last_3month_sum_revenue', 'last_3month_revenue',
       'last_6month_avg_revenue', 'last_6month_sum_revenue',
       'last_6month_revenue', 'item_last_6month_avg_revenue_compared',
       'last_6month_deviation', 'last_month_item_price',
       'item_last_month_item_price_compared',
       'last_3month_avg_item_price', 'last_3month_item_price',
       'last_6month_avg_item_price', 'last_6month_item_price',
       'item_last_6month_avg_item_price_compared',
       'last_3month_deviation_itemprice',
       'last_6month_deviation_itemprice', 'item_category_id', 'is_v31',
       'is_v32', 'is_v33', 'is_v34', 'is_v35', 'is_v36', 'is_v37',
       'is_v38', 'is_v39', 'is_v40', 'is_v41', 'is_v42', 'is_v43',
       'is_v44', 'is_v45', 'is_v46', 'is_v47', 'is_v48', 'is_v49',
       'is_v50', 'is_v51', 'is_v52', 'is_v53', 'is_v54', 'is_v55',
       'is_v56', 'is_v57', 'is_v58', 'is_v59', 'is_v60', 'is_v61',
       'is_v62', 'is_v63', 'is_v64', 'is_v65', 'is_v66', 'is_v67',
       'is_v68', 'is_v69', 'is_v70', 'is_v71', 'is_v72', 'is_v73',
       'is_v74', 'is_v75', 'is_v76', 'is_v77', 'is_v78', 'is_v79',
       'is_v80', 'is_v81', 'is_v82', 'is_v83', 'is_v84', 'is_v85',
       'is_v86', 'is_v87', 'is_v88', 'is_v89', 'is_v90', 'is_v91',
       'is_v92', 'is_v93', 'is_v94', 'is_v95', 'is_v96', 'is_v97',
       'is_v98', 'is_v99', 'is_v100', 'is_v101', 'is_v102', 'is_v103',
       'is_v104', 'is_v105', 'is_v106', 'is_v107', 'is_v108', 'is_v109',
       'is_v110', 'is_v111', 'is_v112', 'is_v113', 'is_v114', 'is_v115',
       'is_v116', 'is_v117', 'is_v118', 'is_v119', 'is_v120', 'is_v121',
       'is_v122', 'is_v123', 'is_v124', 'is_v125', 'is_v126', 'is_v127',
       'is_v128', 'is_v129', 'is_v130', 'is_v131', 'is_v132', 'is_v133',
       'is_v134', 'is_v135', 'is_v136', 'is_v137', 'is_v138', 'is_v139',
       'is_v140', 'is_v141', 'is_v142', 'is_v143', 'is_v144', 'is_v145',
       'is_v146', 'is_v147', 'is_v148', 'is_v149', 'is_v150', 'is_v151',
       'is_v152', 'is_v153', 'is_v154', 'is_v155', 'is_v156', 'is_v157',
       'is_v158', 'is_v159', 'is_v160', 'is_v161', 'is_v162', 'is_v163',
       'is_v164', 'is_v165', 'is_v166', 'is_v167', 'is_v168', 'is_v169',
       'is_v170', 'is_v171', 'is_v172', 'is_v173', 'is_v174', 'is_v175',
       'is_v176', 'is_v177', 'is_v178', 'is_v179', 'is_v180',
       'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231', 
       'store_size', 'store_size_per_month', 'trend', 'yhat_lower',
       'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms',
       'additive_terms_lower', 'additive_terms_upper', 'weekly',
       'weekly_lower', 'weekly_upper', 'yearly', 'yearly_lower',
       'yearly_upper', 'yhat', 'month', 'year']

In [7]:
x_test.columns=['date_block_num', 'shop_id', 'item_id', 'target', 'item_price',
       'last_month_revenue', 'last_2month_avg_revenue',
       'last_3month_avg_revenue', 'last_3month_deviation',
       'item_is_featured_global', 'item_last_month_revenue_compared',
       'item_last_2month_avg_revenue_compared',
       'item_last_3month_avg_revenue_compared', 'shop_is_opening',
       'shop_was_closed_month_before', 'shop_was_closed_6_months_before',
       'last_2month_revenue', 'item_last_2month_revenue_compared',
       'last_3month_sum_revenue', 'last_3month_revenue',
       'last_6month_avg_revenue', 'last_6month_sum_revenue',
       'last_6month_revenue', 'item_last_6month_avg_revenue_compared',
       'last_6month_deviation', 'last_month_item_price',
       'item_last_month_item_price_compared',
       'last_3month_avg_item_price', 'last_3month_item_price',
       'last_6month_avg_item_price', 'last_6month_item_price',
       'item_last_6month_avg_item_price_compared',
       'last_3month_deviation_itemprice',
       'last_6month_deviation_itemprice', 'item_category_id', 'is_v31',
       'is_v32', 'is_v33', 'is_v34', 'is_v35', 'is_v36', 'is_v37',
       'is_v38', 'is_v39', 'is_v40', 'is_v41', 'is_v42', 'is_v43',
       'is_v44', 'is_v45', 'is_v46', 'is_v47', 'is_v48', 'is_v49',
       'is_v50', 'is_v51', 'is_v52', 'is_v53', 'is_v54', 'is_v55',
       'is_v56', 'is_v57', 'is_v58', 'is_v59', 'is_v60', 'is_v61',
       'is_v62', 'is_v63', 'is_v64', 'is_v65', 'is_v66', 'is_v67',
       'is_v68', 'is_v69', 'is_v70', 'is_v71', 'is_v72', 'is_v73',
       'is_v74', 'is_v75', 'is_v76', 'is_v77', 'is_v78', 'is_v79',
       'is_v80', 'is_v81', 'is_v82', 'is_v83', 'is_v84', 'is_v85',
       'is_v86', 'is_v87', 'is_v88', 'is_v89', 'is_v90', 'is_v91',
       'is_v92', 'is_v93', 'is_v94', 'is_v95', 'is_v96', 'is_v97',
       'is_v98', 'is_v99', 'is_v100', 'is_v101', 'is_v102', 'is_v103',
       'is_v104', 'is_v105', 'is_v106', 'is_v107', 'is_v108', 'is_v109',
       'is_v110', 'is_v111', 'is_v112', 'is_v113', 'is_v114', 'is_v115',
       'is_v116', 'is_v117', 'is_v118', 'is_v119', 'is_v120', 'is_v121',
       'is_v122', 'is_v123', 'is_v124', 'is_v125', 'is_v126', 'is_v127',
       'is_v128', 'is_v129', 'is_v130', 'is_v131', 'is_v132', 'is_v133',
       'is_v134', 'is_v135', 'is_v136', 'is_v137', 'is_v138', 'is_v139',
       'is_v140', 'is_v141', 'is_v142', 'is_v143', 'is_v144', 'is_v145',
       'is_v146', 'is_v147', 'is_v148', 'is_v149', 'is_v150', 'is_v151',
       'is_v152', 'is_v153', 'is_v154', 'is_v155', 'is_v156', 'is_v157',
       'is_v158', 'is_v159', 'is_v160', 'is_v161', 'is_v162', 'is_v163',
       'is_v164', 'is_v165', 'is_v166', 'is_v167', 'is_v168', 'is_v169',
       'is_v170', 'is_v171', 'is_v172', 'is_v173', 'is_v174', 'is_v175',
       'is_v176', 'is_v177', 'is_v178', 'is_v179', 'is_v180',
       'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231', 
       'store_size', 'store_size_per_month', 'trend', 'yhat_lower',
       'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms',
       'additive_terms_lower', 'additive_terms_upper', 'weekly',
       'weekly_lower', 'weekly_upper', 'yearly', 'yearly_lower',
       'yearly_upper', 'yhat', 'month', 'year']

In [8]:
def lgb_modelfit_nocv(params, dtrain, dvalid, predictors, target='target', objective='binary', metrics='auc',
                 feval=None, early_stopping_rounds=50, num_boost_round=3000, verbose_eval=10, categorical_features=None):
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': objective,
        'metric':metrics,
        'learning_rate': 0.01,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
        'max_depth': -1,  # -1 means no limit
        'min_child_samples': 20,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.6,  # Subsample ratio of the training instance.
        'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 4,
        'verbose': 0,
        'metric':metrics
    }

    lgb_params.update(params)

    print("preparing validation datasets")

    xgtrain = lgbm.Dataset(dtrain[predictors].values, label=dtrain[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    xgvalid = lgbm.Dataset(dvalid[predictors].values, label=dvalid[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )

    evals_results = {}

    bst1 = lgbm.train(lgb_params,
                     xgtrain,
                     valid_sets=[xgtrain, xgvalid],
                     valid_names=['train','valid'],
                     evals_result=evals_results,
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10,
                     feval=feval)

    n_estimators = bst1.best_iteration
    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    print(metrics+":", evals_results['valid'][metrics][n_estimators-1])

    return bst1

In [9]:
categorical=[ 'item_is_featured_global','shop_is_opening',
       'shop_was_closed_month_before', 'shop_was_closed_6_months_before', 'is_v31',
       'is_v32', 'is_v33', 'is_v34', 'is_v35', 'is_v36', 'is_v37',
       'is_v38', 'is_v39', 'is_v40', 'is_v41', 'is_v42', 'is_v43',
       'is_v44', 'is_v45', 'is_v46', 'is_v47', 'is_v48', 'is_v49',
       'is_v50', 'is_v51', 'is_v52', 'is_v53', 'is_v54', 'is_v55',
       'is_v56', 'is_v57', 'is_v58', 'is_v59', 'is_v60', 'is_v61',
       'is_v62', 'is_v63', 'is_v64', 'is_v65', 'is_v66', 'is_v67',
       'is_v68', 'is_v69', 'is_v70', 'is_v71', 'is_v72', 'is_v73',
       'is_v74', 'is_v75', 'is_v76', 'is_v77', 'is_v78', 'is_v79',
       'is_v80', 'is_v81', 'is_v82', 'is_v83', 'is_v84', 'is_v85',
       'is_v86', 'is_v87', 'is_v88', 'is_v89', 'is_v90', 'is_v91',
       'is_v92', 'is_v93', 'is_v94', 'is_v95', 'is_v96', 'is_v97',
       'is_v98', 'is_v99', 'is_v100', 'is_v101', 'is_v102', 'is_v103',
       'is_v104', 'is_v105', 'is_v106', 'is_v107', 'is_v108', 'is_v109',
       'is_v110', 'is_v111', 'is_v112', 'is_v113', 'is_v114', 'is_v115',
       'is_v116', 'is_v117', 'is_v118', 'is_v119', 'is_v120', 'is_v121',
       'is_v122', 'is_v123', 'is_v124', 'is_v125', 'is_v126', 'is_v127',
       'is_v128', 'is_v129', 'is_v130', 'is_v131', 'is_v132', 'is_v133',
       'is_v134', 'is_v135', 'is_v136', 'is_v137', 'is_v138', 'is_v139',
       'is_v140', 'is_v141', 'is_v142', 'is_v143', 'is_v144', 'is_v145',
       'is_v146', 'is_v147', 'is_v148', 'is_v149', 'is_v150', 'is_v151',
       'is_v152', 'is_v153', 'is_v154', 'is_v155', 'is_v156', 'is_v157',
       'is_v158', 'is_v159', 'is_v160', 'is_v161', 'is_v162', 'is_v163',
       'is_v164', 'is_v165', 'is_v166', 'is_v167', 'is_v168', 'is_v169',
       'is_v170', 'is_v171', 'is_v172', 'is_v173', 'is_v174', 'is_v175',
       'is_v176', 'is_v177', 'is_v178', 'is_v179', 'is_v180',
       'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231',
       'store_size', 'store_size_per_month']
pred=['last_month_revenue', 'last_2month_avg_revenue',
       'last_3month_avg_revenue', 'last_3month_deviation',
       'item_is_featured_global', 'item_last_month_revenue_compared',
       'item_last_2month_avg_revenue_compared',
       'item_last_3month_avg_revenue_compared', 'shop_is_opening',
       'shop_was_closed_month_before', 'shop_was_closed_6_months_before',
       'last_2month_revenue', 'item_last_2month_revenue_compared',
       'last_3month_sum_revenue', 'last_3month_revenue',
       'last_6month_avg_revenue', 'last_6month_sum_revenue',
       'last_6month_revenue', 'item_last_6month_avg_revenue_compared',
       'last_6month_deviation', 'last_month_item_price',
       'item_last_month_item_price_compared',
       'last_3month_avg_item_price', 'last_3month_item_price',
       'last_6month_avg_item_price', 'last_6month_item_price',
       'item_last_6month_avg_item_price_compared',
       'last_3month_deviation_itemprice',
       'last_6month_deviation_itemprice', 'is_v31',
       'is_v32', 'is_v33', 'is_v34', 'is_v35', 'is_v36', 'is_v37',
       'is_v38', 'is_v39', 'is_v40', 'is_v41', 'is_v42', 'is_v43',
       'is_v44', 'is_v45', 'is_v46', 'is_v47', 'is_v48', 'is_v49',
       'is_v50', 'is_v51', 'is_v52', 'is_v53', 'is_v54', 'is_v55',
       'is_v56', 'is_v57', 'is_v58', 'is_v59', 'is_v60', 'is_v61',
       'is_v62', 'is_v63', 'is_v64', 'is_v65', 'is_v66', 'is_v67',
       'is_v68', 'is_v69', 'is_v70', 'is_v71', 'is_v72', 'is_v73',
       'is_v74', 'is_v75', 'is_v76', 'is_v77', 'is_v78', 'is_v79',
       'is_v80', 'is_v81', 'is_v82', 'is_v83', 'is_v84', 'is_v85',
       'is_v86', 'is_v87', 'is_v88', 'is_v89', 'is_v90', 'is_v91',
       'is_v92', 'is_v93', 'is_v94', 'is_v95', 'is_v96', 'is_v97',
       'is_v98', 'is_v99', 'is_v100', 'is_v101', 'is_v102', 'is_v103',
       'is_v104', 'is_v105', 'is_v106', 'is_v107', 'is_v108', 'is_v109',
       'is_v110', 'is_v111', 'is_v112', 'is_v113', 'is_v114', 'is_v115',
       'is_v116', 'is_v117', 'is_v118', 'is_v119', 'is_v120', 'is_v121',
       'is_v122', 'is_v123', 'is_v124', 'is_v125', 'is_v126', 'is_v127',
       'is_v128', 'is_v129', 'is_v130', 'is_v131', 'is_v132', 'is_v133',
       'is_v134', 'is_v135', 'is_v136', 'is_v137', 'is_v138', 'is_v139',
       'is_v140', 'is_v141', 'is_v142', 'is_v143', 'is_v144', 'is_v145',
       'is_v146', 'is_v147', 'is_v148', 'is_v149', 'is_v150', 'is_v151',
       'is_v152', 'is_v153', 'is_v154', 'is_v155', 'is_v156', 'is_v157',
       'is_v158', 'is_v159', 'is_v160', 'is_v161', 'is_v162', 'is_v163',
       'is_v164', 'is_v165', 'is_v166', 'is_v167', 'is_v168', 'is_v169',
       'is_v170', 'is_v171', 'is_v172', 'is_v173', 'is_v174', 'is_v175',
       'is_v176', 'is_v177', 'is_v178', 'is_v179', 'is_v180',
       'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231',
       'store_size', 'store_size_per_month', 'trend', 'yhat_lower',
       'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms',
       'additive_terms_lower', 'additive_terms_upper', 'weekly',
       'weekly_lower', 'weekly_upper', 'yearly', 'yearly_lower',
       'yearly_upper', 'yhat']

In [10]:
params={'min_child_samples': 10} ## We'll be setting some parameters at first just to not over grow tree and produce an "overfitted" model

### Notice that I removed some of the variables that could be important in the model:
* item_id, shop_id and item_category_id
> I did this mainly because all of this features are categorical and has different levels at both datasets. This could imply a worst performance at our test dataset

In [11]:
best_lgbm = lgb_modelfit_nocv(params, x_train[pred+['target']], x_test[pred+['target']], predictors=pred, 
                              target='target', objective='regression', metrics='rmse',categorical_features=categorical)

preparing validation datasets


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[10]	train's rmse: 1.44269	valid's rmse: 1.285
[20]	train's rmse: 1.41532	valid's rmse: 1.25772
[30]	train's rmse: 1.38963	valid's rmse: 1.23319
[40]	train's rmse: 1.36776	valid's rmse: 1.21303
[50]	train's rmse: 1.34828	valid's rmse: 1.19548
[60]	train's rmse: 1.33128	valid's rmse: 1.18082
[70]	train's rmse: 1.31606	valid's rmse: 1.16793
[80]	train's rmse: 1.30307	valid's rmse: 1.15717
[90]	train's rmse: 1.28899	valid's rmse: 1.14565
[100]	train's rmse: 1.2782	valid's rmse: 1.1379
[110]	train's rmse: 1.26837	valid's rmse: 1.13114
[120]	train's rmse: 1.26054	valid's rmse: 1.1258
[130]	train's rmse: 1.25288	valid's rmse: 1.12059
[140]	train's rmse: 1.24609	valid's rmse: 1.11608
[150]	train's rmse: 1.24	valid's rmse: 1.11188
[160]	train's rmse: 1.23475	valid's rmse: 1.10904
[170]	train's rmse: 1.22918	valid's rmse: 1.10646
[180]	train's rmse: 1.22357	valid's rmse: 1.10309
[190]	train's rmse: 1.21742	valid's rmse: 1.09962
[200]	

[1640]	train's rmse: 1.05651	valid's rmse: 1.04839
[1650]	train's rmse: 1.05592	valid's rmse: 1.04823
[1660]	train's rmse: 1.05556	valid's rmse: 1.04821
[1670]	train's rmse: 1.05518	valid's rmse: 1.04808
[1680]	train's rmse: 1.05483	valid's rmse: 1.048
[1690]	train's rmse: 1.05429	valid's rmse: 1.04785
[1700]	train's rmse: 1.05385	valid's rmse: 1.04778
[1710]	train's rmse: 1.0534	valid's rmse: 1.04767
[1720]	train's rmse: 1.0531	valid's rmse: 1.04762
[1730]	train's rmse: 1.05285	valid's rmse: 1.04755
[1740]	train's rmse: 1.05252	valid's rmse: 1.04749
[1750]	train's rmse: 1.05213	valid's rmse: 1.0474
[1760]	train's rmse: 1.05163	valid's rmse: 1.04741
[1770]	train's rmse: 1.05131	valid's rmse: 1.04736
[1780]	train's rmse: 1.05092	valid's rmse: 1.0473
[1790]	train's rmse: 1.05064	valid's rmse: 1.04723
[1800]	train's rmse: 1.05034	valid's rmse: 1.04722
[1810]	train's rmse: 1.05001	valid's rmse: 1.04715
[1820]	train's rmse: 1.04966	valid's rmse: 1.04702
[1830]	train's rmse: 1.04939	valid's 

## Trying to overfit our model

In [12]:
params={'min_child_samples': 30,
       'learning_rate':0.01,
        'num_leaves': 50,
        'reg_lambda': 5,
        'reg_alpha': 5,
        'min_split_gain': 10,
        'max_bin': 300
       } 
#del(best_lgbm)
best_lgbm = lgb_modelfit_nocv(params, x_train[pred+['target']], x_test[pred+['target']], predictors=pred, 
                              target='target', objective='regression', metrics='rmse',categorical_features=categorical)

preparing validation datasets
Training until validation scores don't improve for 50 rounds
[10]	train's rmse: 1.44024	valid's rmse: 1.28365
[20]	train's rmse: 1.41125	valid's rmse: 1.25611
[30]	train's rmse: 1.38406	valid's rmse: 1.23161
[40]	train's rmse: 1.36113	valid's rmse: 1.21124
[50]	train's rmse: 1.3408	valid's rmse: 1.19415
[60]	train's rmse: 1.32357	valid's rmse: 1.17912
[70]	train's rmse: 1.3087	valid's rmse: 1.16746
[80]	train's rmse: 1.29479	valid's rmse: 1.15691
[90]	train's rmse: 1.28266	valid's rmse: 1.14752
[100]	train's rmse: 1.27109	valid's rmse: 1.1393
[110]	train's rmse: 1.26113	valid's rmse: 1.13206
[120]	train's rmse: 1.25243	valid's rmse: 1.12654
[130]	train's rmse: 1.24194	valid's rmse: 1.11951
[140]	train's rmse: 1.23415	valid's rmse: 1.11443
[150]	train's rmse: 1.22727	valid's rmse: 1.11063
[160]	train's rmse: 1.21995	valid's rmse: 1.1065
[170]	train's rmse: 1.2142	valid's rmse: 1.10339
[180]	train's rmse: 1.20757	valid's rmse: 1.09913
[190]	train's rmse: 1.2

[1630]	train's rmse: 1.03177	valid's rmse: 1.04822
[1640]	train's rmse: 1.03132	valid's rmse: 1.04816
[1650]	train's rmse: 1.03098	valid's rmse: 1.04808
[1660]	train's rmse: 1.03072	valid's rmse: 1.04808
[1670]	train's rmse: 1.03041	valid's rmse: 1.04812
[1680]	train's rmse: 1.03014	valid's rmse: 1.04805
[1690]	train's rmse: 1.02982	valid's rmse: 1.04807
[1700]	train's rmse: 1.02954	valid's rmse: 1.04797
[1710]	train's rmse: 1.02921	valid's rmse: 1.04783
[1720]	train's rmse: 1.02889	valid's rmse: 1.0478
[1730]	train's rmse: 1.02853	valid's rmse: 1.04772
[1740]	train's rmse: 1.02823	valid's rmse: 1.04755
[1750]	train's rmse: 1.02788	valid's rmse: 1.04754
[1760]	train's rmse: 1.02757	valid's rmse: 1.0474
[1770]	train's rmse: 1.02735	valid's rmse: 1.04736
[1780]	train's rmse: 1.02711	valid's rmse: 1.04731
[1790]	train's rmse: 1.02685	valid's rmse: 1.04719
[1800]	train's rmse: 1.02657	valid's rmse: 1.04712
[1810]	train's rmse: 1.02631	valid's rmse: 1.04703
[1820]	train's rmse: 1.02602	vali

### In this part we'll take advantage of a data leakage and try to fit our model only for the stores presented in the test final dataset!

In [13]:
test=pd.read_csv('test_final.csv')
x_train_v2 = x_train[x_train.shop_id.isin(test.shop_id.unique())]
x_test_v2 = x_test[x_test.shop_id.isin(test.shop_id.unique())]
del(x_train)
del(x_test)

In [14]:
params={}

In [15]:
best_lgbm2 = lgb_modelfit_nocv(params, x_train_v2[pred+['target']], x_test_v2[pred+['target']], predictors=pred, 
                              target='target', objective='regression', metrics='rmse',categorical_features=categorical)

preparing validation datasets
Training until validation scores don't improve for 50 rounds
[10]	train's rmse: 1.42907	valid's rmse: 1.2661
[20]	train's rmse: 1.40157	valid's rmse: 1.2375
[30]	train's rmse: 1.37757	valid's rmse: 1.21439
[40]	train's rmse: 1.35398	valid's rmse: 1.19212
[50]	train's rmse: 1.33446	valid's rmse: 1.17442
[60]	train's rmse: 1.31624	valid's rmse: 1.15807
[70]	train's rmse: 1.30065	valid's rmse: 1.14475
[80]	train's rmse: 1.28545	valid's rmse: 1.13169
[90]	train's rmse: 1.27232	valid's rmse: 1.12135
[100]	train's rmse: 1.26192	valid's rmse: 1.11319
[110]	train's rmse: 1.25219	valid's rmse: 1.10541
[120]	train's rmse: 1.24432	valid's rmse: 1.09919
[130]	train's rmse: 1.23548	valid's rmse: 1.09225
[140]	train's rmse: 1.22862	valid's rmse: 1.08739
[150]	train's rmse: 1.22168	valid's rmse: 1.0825
[160]	train's rmse: 1.21519	valid's rmse: 1.07785
[170]	train's rmse: 1.21032	valid's rmse: 1.07497
[180]	train's rmse: 1.20509	valid's rmse: 1.07203
[190]	train's rmse: 1

[1630]	train's rmse: 1.03598	valid's rmse: 1.00919
[1640]	train's rmse: 1.03565	valid's rmse: 1.00899
[1650]	train's rmse: 1.03523	valid's rmse: 1.00889
[1660]	train's rmse: 1.03487	valid's rmse: 1.00887
[1670]	train's rmse: 1.03452	valid's rmse: 1.0088
[1680]	train's rmse: 1.03414	valid's rmse: 1.00869
[1690]	train's rmse: 1.03377	valid's rmse: 1.00855
[1700]	train's rmse: 1.03349	valid's rmse: 1.00851
[1710]	train's rmse: 1.03311	valid's rmse: 1.00843
[1720]	train's rmse: 1.03281	valid's rmse: 1.00832
[1730]	train's rmse: 1.0324	valid's rmse: 1.00812
[1740]	train's rmse: 1.03214	valid's rmse: 1.00806
[1750]	train's rmse: 1.03178	valid's rmse: 1.00795
[1760]	train's rmse: 1.03147	valid's rmse: 1.00784
[1770]	train's rmse: 1.03123	valid's rmse: 1.00776
[1780]	train's rmse: 1.03101	valid's rmse: 1.00769
[1790]	train's rmse: 1.03083	valid's rmse: 1.00765
[1800]	train's rmse: 1.03048	valid's rmse: 1.00755
[1810]	train's rmse: 1.03017	valid's rmse: 1.00744
[1820]	train's rmse: 1.02992	vali

In [16]:
params={'min_child_samples': 30,
       'learning_rate':0.01,
        'num_leaves': 80,
        'reg_lambda': 2,
        'max_bin': 300
       } 
#del(best_lgbm)
best_lgbm2 = lgb_modelfit_nocv(params, x_train_v2[pred+['target']], x_test_v2[pred+['target']], predictors=pred, 
                              target='target', objective='regression', metrics='rmse',categorical_features=categorical)

preparing validation datasets
Training until validation scores don't improve for 50 rounds
[10]	train's rmse: 1.42499	valid's rmse: 1.26418
[20]	train's rmse: 1.39399	valid's rmse: 1.23392
[30]	train's rmse: 1.36662	valid's rmse: 1.20925
[40]	train's rmse: 1.33916	valid's rmse: 1.1854
[50]	train's rmse: 1.31642	valid's rmse: 1.16639
[60]	train's rmse: 1.29526	valid's rmse: 1.14856
[70]	train's rmse: 1.27642	valid's rmse: 1.13407
[80]	train's rmse: 1.25869	valid's rmse: 1.11991
[90]	train's rmse: 1.24299	valid's rmse: 1.10869
[100]	train's rmse: 1.2304	valid's rmse: 1.09988
[110]	train's rmse: 1.21909	valid's rmse: 1.09167
[120]	train's rmse: 1.20977	valid's rmse: 1.08534
[130]	train's rmse: 1.19956	valid's rmse: 1.07789
[140]	train's rmse: 1.1915	valid's rmse: 1.07294
[150]	train's rmse: 1.18297	valid's rmse: 1.06757
[160]	train's rmse: 1.17528	valid's rmse: 1.06306
[170]	train's rmse: 1.16926	valid's rmse: 1.05999
[180]	train's rmse: 1.16308	valid's rmse: 1.05698
[190]	train's rmse: 1

[1620]	train's rmse: 0.970496	valid's rmse: 1.00197
[1630]	train's rmse: 0.970162	valid's rmse: 1.00192
[1640]	train's rmse: 0.96972	valid's rmse: 1.00181
[1650]	train's rmse: 0.969351	valid's rmse: 1.00175
[1660]	train's rmse: 0.96895	valid's rmse: 1.00176
[1670]	train's rmse: 0.968589	valid's rmse: 1.00174
[1680]	train's rmse: 0.968229	valid's rmse: 1.00166
[1690]	train's rmse: 0.967848	valid's rmse: 1.00158
[1700]	train's rmse: 0.967529	valid's rmse: 1.00153
[1710]	train's rmse: 0.967166	valid's rmse: 1.00142
[1720]	train's rmse: 0.966763	valid's rmse: 1.00136
[1730]	train's rmse: 0.966364	valid's rmse: 1.00119
[1740]	train's rmse: 0.966034	valid's rmse: 1.00115
[1750]	train's rmse: 0.965704	valid's rmse: 1.00105
[1760]	train's rmse: 0.965342	valid's rmse: 1.00096
[1770]	train's rmse: 0.964966	valid's rmse: 1.00092
[1780]	train's rmse: 0.964661	valid's rmse: 1.00085
[1790]	train's rmse: 0.964346	valid's rmse: 1.00076
[1800]	train's rmse: 0.964022	valid's rmse: 1.00066
[1810]	train's

In [17]:
params={
       'learning_rate':0.01,
        'num_leaves': 80,
        'reg_lambda': 10,
        'min_split_gain': 10,
        'max_bin': 300
       }
best_lgbm3 = lgb_modelfit_nocv(params, x_train_v2[pred+['target']], x_test_v2[pred+['target']], predictors=pred, 
                              target='target', objective='regression', metrics='rmse',categorical_features=categorical)

preparing validation datasets
Training until validation scores don't improve for 50 rounds
[10]	train's rmse: 1.42557	valid's rmse: 1.26433
[20]	train's rmse: 1.39527	valid's rmse: 1.23476
[30]	train's rmse: 1.36843	valid's rmse: 1.21039
[40]	train's rmse: 1.34144	valid's rmse: 1.18671
[50]	train's rmse: 1.31914	valid's rmse: 1.16798
[60]	train's rmse: 1.29827	valid's rmse: 1.15004
[70]	train's rmse: 1.2798	valid's rmse: 1.13544
[80]	train's rmse: 1.2624	valid's rmse: 1.12151
[90]	train's rmse: 1.24689	valid's rmse: 1.11039
[100]	train's rmse: 1.23445	valid's rmse: 1.10164
[110]	train's rmse: 1.22331	valid's rmse: 1.09339
[120]	train's rmse: 1.21396	valid's rmse: 1.08691
[130]	train's rmse: 1.20381	valid's rmse: 1.07945
[140]	train's rmse: 1.19585	valid's rmse: 1.07455
[150]	train's rmse: 1.18748	valid's rmse: 1.06944
[160]	train's rmse: 1.17991	valid's rmse: 1.06507
[170]	train's rmse: 1.17397	valid's rmse: 1.06189
[180]	train's rmse: 1.1678	valid's rmse: 1.05878
[190]	train's rmse: 1

[1630]	train's rmse: 0.977269	valid's rmse: 0.999712
[1640]	train's rmse: 0.97682	valid's rmse: 0.999557
[1650]	train's rmse: 0.976452	valid's rmse: 0.999549
[1660]	train's rmse: 0.976115	valid's rmse: 0.999522
[1670]	train's rmse: 0.975784	valid's rmse: 0.999474
[1680]	train's rmse: 0.975481	valid's rmse: 0.999446
[1690]	train's rmse: 0.975141	valid's rmse: 0.99932
[1700]	train's rmse: 0.974836	valid's rmse: 0.999235
[1710]	train's rmse: 0.974462	valid's rmse: 0.999135
[1720]	train's rmse: 0.974145	valid's rmse: 0.9991
[1730]	train's rmse: 0.973871	valid's rmse: 0.999031
[1740]	train's rmse: 0.973531	valid's rmse: 0.998956
[1750]	train's rmse: 0.973214	valid's rmse: 0.998883
[1760]	train's rmse: 0.972952	valid's rmse: 0.998864
[1770]	train's rmse: 0.972646	valid's rmse: 0.998883
[1780]	train's rmse: 0.972365	valid's rmse: 0.998862
[1790]	train's rmse: 0.972028	valid's rmse: 0.998776
[1800]	train's rmse: 0.97168	valid's rmse: 0.998759
[1810]	train's rmse: 0.971269	valid's rmse: 0.99860

In [18]:
test.columns=['Unnamed: 0', 'ID', 'shop_id', 'item_id', 'date_block_num',
       'revenue', 'item_price', 'last_month_revenue',
       'item_last_month_revenue_compared', 'shop_was_closed_month_before',
       'last_2month_revenue', 'last_2month_avg_revenue',
       'item_last_2month_avg_revenue_compared',
       'item_last_2month_revenue_compared', 'last_3month_avg_revenue',
       'last_3month_sum_revenue', 'last_3month_revenue',
       'item_last_3month_avg_revenue_compared', 'last_6month_avg_revenue',
       'last_6month_sum_revenue', 'last_6month_revenue',
       'item_last_6month_avg_revenue_compared',
       'shop_was_closed_6_months_before', 'last_month_item_price',
       'item_last_month_item_price_compared',
       'last_3month_avg_item_price', 'last_3month_item_price',
       'last_6month_avg_item_price', 'last_6month_item_price',
       'item_last_6month_avg_item_price_compared', 'Unnamed: 0.1',
       'item_category_id', 'is_v31', 'is_v32', 'is_v33', 'is_v34',
       'is_v35', 'is_v36', 'is_v37', 'is_v38', 'is_v39', 'is_v40',
       'is_v41', 'is_v42', 'is_v43', 'is_v44', 'is_v45', 'is_v46',
       'is_v47', 'is_v48', 'is_v49', 'is_v50', 'is_v51', 'is_v52',
       'is_v53', 'is_v54', 'is_v55', 'is_v56', 'is_v57', 'is_v58',
       'is_v59', 'is_v60', 'is_v61', 'is_v62', 'is_v63', 'is_v64',
       'is_v65', 'is_v66', 'is_v67', 'is_v68', 'is_v69', 'is_v70',
       'is_v71', 'is_v72', 'is_v73', 'is_v74', 'is_v75', 'is_v76',
       'is_v77', 'is_v78', 'is_v79', 'is_v80', 'is_v81', 'is_v82',
       'is_v83', 'is_v84', 'is_v85', 'is_v86', 'is_v87', 'is_v88',
       'is_v89', 'is_v90', 'is_v91', 'is_v92', 'is_v93', 'is_v94',
       'is_v95', 'is_v96', 'is_v97', 'is_v98', 'is_v99', 'is_v100',
       'is_v101', 'is_v102', 'is_v103', 'is_v104', 'is_v105', 'is_v106',
       'is_v107', 'is_v108', 'is_v109', 'is_v110', 'is_v111', 'is_v112',
       'is_v113', 'is_v114', 'is_v115', 'is_v116', 'is_v117', 'is_v118',
       'is_v119', 'is_v120', 'is_v121', 'is_v122', 'is_v123', 'is_v124',
       'is_v125', 'is_v126', 'is_v127', 'is_v128', 'is_v129', 'is_v130',
       'is_v131', 'is_v132', 'is_v133', 'is_v134', 'is_v135', 'is_v136',
       'is_v137', 'is_v138', 'is_v139', 'is_v140', 'is_v141', 'is_v142',
       'is_v143', 'is_v144', 'is_v145', 'is_v146', 'is_v147', 'is_v148',
       'is_v149', 'is_v150', 'is_v151', 'is_v152', 'is_v153', 'is_v154',
       'is_v155', 'is_v156', 'is_v157', 'is_v158', 'is_v159', 'is_v160',
       'is_v161', 'is_v162', 'is_v163', 'is_v164', 'is_v165', 'is_v166',
       'is_v167', 'is_v168', 'is_v169', 'is_v170', 'is_v171', 'is_v172',
       'is_v173', 'is_v174', 'is_v175', 'is_v176', 'is_v177', 'is_v178',
       'is_v179', 'is_v180', 'Unnamed: 0_2','v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231',
       'store_size', 'store_size_per_month', 'Unnamed: 0_2.1', 'trend',
       'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
       'weekly', 'weekly_lower', 'weekly_upper', 'yearly', 'yearly_lower',
       'yearly_upper', 'yhat', 'last_3month_deviation',
       'last_6month_deviation', 'last_3month_deviation_itemprice',
       'last_6month_deviation_itemprice', 'item_is_featured_global',
       'shop_is_opening']

In [22]:
test['item_cnt_month']=best_lgbm.predict(test[pred])
test[['ID','item_cnt_month']].to_csv('submission_v50.csv',index_label='ID', index=False)

In [23]:
test['item_cnt_month']=best_lgbm2.predict(test[pred])
test[['ID','item_cnt_month']].to_csv('submission_v51.csv',index_label='ID', index=False)

In [24]:
test['item_cnt_month']=best_lgbm3.predict(test[pred])
test[['ID','item_cnt_month']].to_csv('submission_v52.csv',index_label='ID', index=False)

In [ ]:
#40: 1.135095 and 1.119995
#41: 1.121916 and 1.108126

In [ ]:
del(best_lgbm)
del(best_lgbm2)
params={
       'learning_rate':0.01,
        'num_leaves': 80,
        'reg_lambda': 10,
        'min_split_gain': 10,
        'max_bin': 400
       }
best_lgbm3 = lgb_modelfit_nocv(params, x_train_v2[pred+['target']], x_test_v2[pred+['target']], predictors=pred, 
                              target='target', objective='regression', metrics='rmse',categorical_features=categorical, num_boost_round=1000)
test['item_cnt_month']=best_lgbm3.predict(test[pred])
test[['ID','item_cnt_month']].to_csv('submission_v53.csv',index_label='ID', index=False)

preparing validation datasets
Training until validation scores don't improve for 50 rounds
[10]	train's rmse: 1.42561	valid's rmse: 1.26445
[20]	train's rmse: 1.39526	valid's rmse: 1.23485
[30]	train's rmse: 1.36843	valid's rmse: 1.21034
[40]	train's rmse: 1.34142	valid's rmse: 1.18668
[50]	train's rmse: 1.31909	valid's rmse: 1.16789
[60]	train's rmse: 1.29826	valid's rmse: 1.14995
[70]	train's rmse: 1.27978	valid's rmse: 1.13547
[80]	train's rmse: 1.26231	valid's rmse: 1.12138
[90]	train's rmse: 1.2468	valid's rmse: 1.11017
[100]	train's rmse: 1.23432	valid's rmse: 1.10136
[110]	train's rmse: 1.22319	valid's rmse: 1.09307
[120]	train's rmse: 1.21384	valid's rmse: 1.08664
[130]	train's rmse: 1.20372	valid's rmse: 1.07917
[140]	train's rmse: 1.19576	valid's rmse: 1.07422
[150]	train's rmse: 1.18748	valid's rmse: 1.06908
[160]	train's rmse: 1.17993	valid's rmse: 1.0646
[170]	train's rmse: 1.17396	valid's rmse: 1.06139
[180]	train's rmse: 1.16779	valid's rmse: 1.05824
[190]	train's rmse: 

In [ ]:
#42 Vanila: 1.116858 and 1.100994

#43 Max trees set to 1000 and maxbins increased:: 1.077249 and 1.062930

# So let's sum up some stuff:
> Overfitted tree with some regularization had the best performance in our train and validation but performed poorly on the test set. One possible reason is that the number of trees overfitted dramatically. 
<br><br>
> The best results in test set was not necessairly the best in train and valid. We'll do another tweek in our dataset and check if the test RMSE improves!

In [ ]:
test['month']='11'
test['year']='2015'
categorical=[ 'item_is_featured_global','shop_is_opening',
       'shop_was_closed_month_before', 'shop_was_closed_6_months_before', 'is_v31',
       'is_v32', 'is_v33', 'is_v34', 'is_v35', 'is_v36', 'is_v37',
       'is_v38', 'is_v39', 'is_v40', 'is_v41', 'is_v42', 'is_v43',
       'is_v44', 'is_v45', 'is_v46', 'is_v47', 'is_v48', 'is_v49',
       'is_v50', 'is_v51', 'is_v52', 'is_v53', 'is_v54', 'is_v55',
       'is_v56', 'is_v57', 'is_v58', 'is_v59', 'is_v60', 'is_v61',
       'is_v62', 'is_v63', 'is_v64', 'is_v65', 'is_v66', 'is_v67',
       'is_v68', 'is_v69', 'is_v70', 'is_v71', 'is_v72', 'is_v73',
       'is_v74', 'is_v75', 'is_v76', 'is_v77', 'is_v78', 'is_v79',
       'is_v80', 'is_v81', 'is_v82', 'is_v83', 'is_v84', 'is_v85',
       'is_v86', 'is_v87', 'is_v88', 'is_v89', 'is_v90', 'is_v91',
       'is_v92', 'is_v93', 'is_v94', 'is_v95', 'is_v96', 'is_v97',
       'is_v98', 'is_v99', 'is_v100', 'is_v101', 'is_v102', 'is_v103',
       'is_v104', 'is_v105', 'is_v106', 'is_v107', 'is_v108', 'is_v109',
       'is_v110', 'is_v111', 'is_v112', 'is_v113', 'is_v114', 'is_v115',
       'is_v116', 'is_v117', 'is_v118', 'is_v119', 'is_v120', 'is_v121',
       'is_v122', 'is_v123', 'is_v124', 'is_v125', 'is_v126', 'is_v127',
       'is_v128', 'is_v129', 'is_v130', 'is_v131', 'is_v132', 'is_v133',
       'is_v134', 'is_v135', 'is_v136', 'is_v137', 'is_v138', 'is_v139',
       'is_v140', 'is_v141', 'is_v142', 'is_v143', 'is_v144', 'is_v145',
       'is_v146', 'is_v147', 'is_v148', 'is_v149', 'is_v150', 'is_v151',
       'is_v152', 'is_v153', 'is_v154', 'is_v155', 'is_v156', 'is_v157',
       'is_v158', 'is_v159', 'is_v160', 'is_v161', 'is_v162', 'is_v163',
       'is_v164', 'is_v165', 'is_v166', 'is_v167', 'is_v168', 'is_v169',
       'is_v170', 'is_v171', 'is_v172', 'is_v173', 'is_v174', 'is_v175',
       'is_v176', 'is_v177', 'is_v178', 'is_v179', 'is_v180',
       'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231',
       'store_size', 'store_size_per_month', 'month', 'year']
pred=['last_month_revenue', 'last_2month_avg_revenue',
       'last_3month_avg_revenue', 'last_3month_deviation',
       'item_is_featured_global', 'item_last_month_revenue_compared',
       'item_last_2month_avg_revenue_compared',
       'item_last_3month_avg_revenue_compared', 'shop_is_opening',
       'shop_was_closed_month_before', 'shop_was_closed_6_months_before',
       'last_2month_revenue', 'item_last_2month_revenue_compared',
       'last_3month_sum_revenue', 'last_3month_revenue',
       'last_6month_avg_revenue', 'last_6month_sum_revenue',
       'last_6month_revenue', 'item_last_6month_avg_revenue_compared',
       'last_6month_deviation', 'last_month_item_price',
       'item_last_month_item_price_compared',
       'last_3month_avg_item_price', 'last_3month_item_price',
       'last_6month_avg_item_price', 'last_6month_item_price',
       'item_last_6month_avg_item_price_compared',
       'last_3month_deviation_itemprice',
       'last_6month_deviation_itemprice', 'is_v31',
       'is_v32', 'is_v33', 'is_v34', 'is_v35', 'is_v36', 'is_v37',
       'is_v38', 'is_v39', 'is_v40', 'is_v41', 'is_v42', 'is_v43',
       'is_v44', 'is_v45', 'is_v46', 'is_v47', 'is_v48', 'is_v49',
       'is_v50', 'is_v51', 'is_v52', 'is_v53', 'is_v54', 'is_v55',
       'is_v56', 'is_v57', 'is_v58', 'is_v59', 'is_v60', 'is_v61',
       'is_v62', 'is_v63', 'is_v64', 'is_v65', 'is_v66', 'is_v67',
       'is_v68', 'is_v69', 'is_v70', 'is_v71', 'is_v72', 'is_v73',
       'is_v74', 'is_v75', 'is_v76', 'is_v77', 'is_v78', 'is_v79',
       'is_v80', 'is_v81', 'is_v82', 'is_v83', 'is_v84', 'is_v85',
       'is_v86', 'is_v87', 'is_v88', 'is_v89', 'is_v90', 'is_v91',
       'is_v92', 'is_v93', 'is_v94', 'is_v95', 'is_v96', 'is_v97',
       'is_v98', 'is_v99', 'is_v100', 'is_v101', 'is_v102', 'is_v103',
       'is_v104', 'is_v105', 'is_v106', 'is_v107', 'is_v108', 'is_v109',
       'is_v110', 'is_v111', 'is_v112', 'is_v113', 'is_v114', 'is_v115',
       'is_v116', 'is_v117', 'is_v118', 'is_v119', 'is_v120', 'is_v121',
       'is_v122', 'is_v123', 'is_v124', 'is_v125', 'is_v126', 'is_v127',
       'is_v128', 'is_v129', 'is_v130', 'is_v131', 'is_v132', 'is_v133',
       'is_v134', 'is_v135', 'is_v136', 'is_v137', 'is_v138', 'is_v139',
       'is_v140', 'is_v141', 'is_v142', 'is_v143', 'is_v144', 'is_v145',
       'is_v146', 'is_v147', 'is_v148', 'is_v149', 'is_v150', 'is_v151',
       'is_v152', 'is_v153', 'is_v154', 'is_v155', 'is_v156', 'is_v157',
       'is_v158', 'is_v159', 'is_v160', 'is_v161', 'is_v162', 'is_v163',
       'is_v164', 'is_v165', 'is_v166', 'is_v167', 'is_v168', 'is_v169',
       'is_v170', 'is_v171', 'is_v172', 'is_v173', 'is_v174', 'is_v175',
       'is_v176', 'is_v177', 'is_v178', 'is_v179', 'is_v180',
       'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231',
       'store_size', 'store_size_per_month', 'trend', 'yhat_lower',
       'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms',
       'additive_terms_lower', 'additive_terms_upper', 'weekly',
       'weekly_lower', 'weekly_upper', 'yearly', 'yearly_lower',
       'yearly_upper', 'yhat', 'month', 'year']

In [ ]:
#44 1.210147 and 1.194534